## 문제 2-4 : 여행 계획 분석기

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint 

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [6]:
# 1. 출력 구조 정의 (여행지, 기간, 예산, 추천도, 주요 활동)
response_schemas = [
    ResponseSchema(name="destination", description="여행지 (예: 부산)"),
    ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일, 5일)"),
    ResponseSchema(name="budget", description="총 여행 예산 (예: 30만원, 100달러)"),
    ResponseSchema(name="rating", description="여행 추천도 (1점에서 5점 사이의 정수)"),
    ResponseSchema(name="activities", description="여행 중 수행한 주요 활동 리스트 (최대 5가지)")
]

# 2. 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

# 3. 프롬프트 템플릿
template = """
당신은 여행 계획 및 후기 분석 전문가입니다.
다음 여행 텍스트에서 주요 정보를 추출하고, 요청된 형식에 맞춰 구조화해주세요.

여행 텍스트: "{travel_text}"

{format_instructions}

중요 규칙:
1. 'rating'은 1점부터 5점 사이의 정수 값으로만 출력하세요.
2. 'activities'는 리스트 형태로 출력하며, 텍스트에서 언급된 주요 활동만 포함하세요.
3. 텍스트에 없는 정보는 비워두거나 '알 수 없음' 등으로 표시할 수 있습니다.
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 4. 모델 초기화 (Groq API 사용)
# Groq API 키는 환경 변수 GROQ_API_KEY로 설정해야 합니다.
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

# 5. 테스트 리뷰 데이터
travel_text_example = "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."

# 6. 체인 구성 및 실행
chain = prompt | model | parser

try:
    output = chain.invoke({"travel_text": travel_text_example})

    # 7. 결과 출력 (Pretty Print)
    pprint(output)

except Exception as e:
    print(f"정보 추출 중 오류 발생: {e}")
    print("Groq API 키가 올바르게 설정되었는지 확인해 주세요.")

print("-" * 30)

{'activities': ['해운대 바다구경', '자갈치시장에서 회 먹기', '감천문화마을 구경'],
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박3일',
 'rating': '4'}
------------------------------
